# A/B Testing

There's only so much you can do with cross validation and learning curves to test the generalization performance of your latest "superfantastic" model. Additionally, there are some approaches for which backtesting is either infeasible or impossible. At some point you're going to have to let the model out into the wild and see what it can really do. But should you just shelve your current model? Should you unleash this superfantastic model on all your customers? Probably not. Instead you should conduct an A/B test.

An A/B test is not unlike the process used in medicine for testing new treatments. In the typical medical situation, there's a current treatment and standards of care. A company might come up with a new drug. Doctors don't simply start prescribing the new drug. Instead, there are randomized clinical trials. A drug company will recruit a number of patients and randomly assign them to two groups: the "control" group and the "treatment" group. Control will get the existing treatment (or a placebo if there is no current treatment) and Treatment will get the new drug. At the end of the trial, the statistics are calculated (you do have an evaluation metric, right?) and then we're back in familiar territory: do we have a *meaningful* difference (in medicine, defined as a clinically significant result) and do we have a believable difference--we do statistical inference.

We can do the same thing for, really, any process change. Suppose we send out a marketing letter every year to raise funds for our non-profit. This year we want to see if adding a hand written note increases the response rate and the contribution amount. We can take our donor list, randomize it and send one-half the regular fundraising letter and the other half the fundraising letter with the hand written note. We can tally the results and see if the differences are meaningful and believable.

I bring up this entirely non-computer example because this might be a situation that you run into when doing data science. The actual process may be entirely manual, old-school snail mail but you will be asked to structure and evaluate the experiment.

In general, however, we are more likely to be working with new and improved models and A/B testing these models often requires a complicated infrastructure. Suppose we are send our customers emails every month to let them know what's on sale. We currently pick the 20 most popular items but we have a good idea that if we could personalize the email to cover the kinds of things each customer is more likely to buy, we will generate more revenue. We spend a few months developing a personalization model and decide we want to A/B test it.

Before we can even do that, we need an infrastructure that:

1. keeps track of every email sent and whether or not that person opened it (email services can do this; you may need to download the information through their API).
2. assigns each customer into either control or treatment (A or B) and keeps track of it.
3. can tie an email address to a customer purchase (easy on a website, not as easy in a brick-and-mortar store).
4. generates the personalized content and keeps track of it (either in advance or on-the-fly depending on the scale and resources).
5. monitor our services so that we know that emails are going out and that personalized content is being generated and used.
6. collects results.

From an experimental design perspective, we have a few complications and options. If we are applying Frequentist inference, we have quite a few guidelines that we *must* follow for the test to be accurate.

1. We must, of course, set a level of significance ahead of time.
2. We must determine the differences in purchase rates and revenue that are meaningful to us; generally, differences that make the development, deployment and maintenance of the model and infrastructure cost effective.
3. Given the minimum difference in purchase rates ("lift") we want to detect, select an appropriate sample size: how many emails do we need to send out before we can detect our minimum lift?
4. We should not, generally, peek at the results (we should not do a test of statistical significance) before the experiment is over. We may need to monitor our test, though, to make sure emails are being sent, etc.
5. We should not, generally, stop the test early because we see favorable results.

There is a "generally" in the last few points because it is possible to build the ability to peek and stop early into your experiment when using Frequentist inference. However, you *must* do it ahead of time. That is, if you find you need 10,000 samples to detect a minimum 5% lift and that it will take four weeks to obtain that many samples, you must build into your test plan the fact that you will "peek" at one week, two weeks, three weeks and, finally, a four weeks. This peeking generally includes having a stronger criteria for statistical significance known as "early stopping".

Why? If you peek at your data at one week and it is statistically significant, how do you know that the data wouldn't turn the other way in another week?

This is actually a problem in clinical trials as well. What if everyone in Group B is healed completely and everyone in Group A (the original treatment) is still dying? Do you wait until the test is over? In general, there are two ways that medical research handle this problem. First, they include "early stopping" criteria in their experimental design or they use Bayesian inference from the start.

And that brings us to Bayesian inference. Under Bayesian inference, you still need to establish meaningful differences (lift) but you can use the estimate of the Posterior distribution at any time to evaluate the believability of your results. If you stop at 10 observations or 10,000 observations that is simply your responsibility. This is kind of daunting actually.

One very effective practice is to be running A/A tests all the time for the situations where you might apply models. An A/A test is pretty straight forward. You do assign people at random to two groups--but you give them both the control! As you analyze this data as if it were an A/B test, you can test your infrastructure for correctness and you can also assess the size of the lift you generally see in that situation. You can then use this as a prior.

A/B Testing brings in a whole new world of concerns.

One of these is data leakages. Your current data on people buying from emails is *biased* based on the way the current email is constructed. For example, if your email includes the 20 most popular items for your store, then the items people see and the purchases they make from the email are biased by that. If you go to use this data for *backtesting* this has an inherent bias towards popularity. If you tried to develop and backtest a model that was based on personal preferences, it might not work as well simply because it has to work against the bias in the data.

The only way around this sort of bias is to pick a small segment of your users and send them a *random* product listing in their email and only use this data for backtesting. (Don't keep those customers in the "random" group from month to month if you want to keep them).

You also have problems with *attribution*. If you send a weekly email and I see Product A in the email for week 1, and I see it in the email for week 2, and I buy it in week 3...which email caused me to buy Product A? Maybe it was both emails? There isn't usually a right or wrong answer, but you must be consistent.

You need to also worry about overlapping and interacting tests. It is usually not a problem if someone is in an email test group and a site test group...but if someone is in a green/blue button test group *and* a dark/light theme test group, that might be a problem.

You also are up against something like the placebo effect called "novelty effects". A treatment might have a significant lift simply because it is new. If you need to run a four week A/B test, and you find that B is better after four weeks, you should consider running a second test.

Finally, you must make sure that your groups do not contain biases. Consider what happens if Group A contains all women and Group B contains all men? Or if Group A contains established clients and Group B contains new acquisitions? As R. A. Fisher was fond of repeating, the single most important thing you can do is "Randomize! Randomize! Randomize!".

In recent years, *Bandit Optimization* has arisen as a possible alternative to A/B testing. In Bandit optimization (which is related to *reinforcement learning*), you assign a model (control or treatment) to a user based on the probability of the model is any good, subject to a minimum. How does that work?

First, let's start with control and treatment. We assign them to users with a 50/50 probability. As results come in, we can adjust those probabilities based on the purchases. If there are more purchases with B, the probability of assigning B increases and the probability of assigning A decreases. If B starts to lose against A, the probability of assigning B starts to decrease. There is usually a floor of some kind, say, five percent.

Now, when we introduce a new model, we renormalize our probabilities to include the new model at, say, 10% so we have A with a 85% probability, B with a 5% probability and C with a 10% probability. If C is really better than A or B, the probability of assigning C will go up.

What does this give you? If it turns out that some fluctuating circumstance made B look better than A, as that circumstance passes, the probability of picking B will go down because it will stop being better. This takes care of the novelty effects. Additionally, there might be seasonal effects. Maybe B did well in the Spring and when Spring ends, the probability of picking B goes down and when Spring returns, the probability of picking B goes up again. All without human intervention.

The downsides to Bandit Optimization is that you need a fairly large amount of traffic for it to work and it doesn't necessarily pick a clear winner. If you need a clear winner, use A/B testing.
